In [ ]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../HOTS'

In [ ]:
import os
import pickle
import datetime
from mix_Network import *

dataset = 'nmnist'
records_path = '../Records'
timestr = datetime.datetime.now().date().isoformat()
timestr = '2020-12-21'
verbose = True

%mkdir -p ../Records
%mkdir -p ../Records/EXP_03_NMNIST

homeo = True
sigma = None
pooling = False
homeinv = False
jitter = False
tau = 5
krnlinit = 'first'
nblay = 3
nbclust = 4

ds = 5
nb_train = 7500//ds
nb_test = 2500//ds

In [ ]:
def get_nmnist(NbTrainingData, NbTestingData):
    def loadev(nbdigit, loader, learningset):
        timout = []
        xout = []
        yout = []
        polout = []
        labout = []
        for i in range(NbTrainingData):
            events, target = next(iter(loader))
            for iev in range(events.shape[1]):
                    timout.append(events[0][iev][learningset.ordering.find("t")].item())
                    xout.append(events[0][iev][learningset.ordering.find("x")].item())
                    yout.append(events[0][iev][learningset.ordering.find("y")].item())
                    polout.append(events[0][iev][learningset.ordering.find("p")].item())
                    labout.append(target.item())

        eventsout = [xout,yout,timout,polout,labout,learningset.sensor_size,2]
        return eventsout
    
    learningset = tonic.datasets.NMNIST(save_to='../Data/',
                                    train=False,
                                    transform=None)
    loader = tonic.datasets.DataLoader(learningset, shuffle=True)
    
    events_train = loadev(NbTrainingData, loader, learningset)
    events_test = loadev(NbTestingData, loader, learningset)
    
    return events_train, events_test

#### Building matrix for logistic regression
def gather_data(events_in,
                tau_cla=150, # characteristic time of a digit
                sample_events=50, sample_space = 1,
                verbose=False, debug=False):
    tau_cla *= 1e3 # to enter tau in ms
    n_events = len(events_in[0])

    c_int = lambda n, d : ((n - 1) // d) + 1
    
    data = np.zeros((c_int(events_in[-2][0], sample_space),
                     c_int(events_in[-2][1], sample_space),
                     events_in[-1])) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    for i_event in range(1, n_events):
        if events_in[2][i_event]<events_in[2][i_event-1]:
            data = np.zeros((c_int(events_in[-2][0], sample_space),
                     c_int(events_in[-2][1], sample_space),
                     events_in[-1])) #tmp data 

        data *= np.exp(-(events_in[2][i_event]-events_in[2][i_event-1])/tau_cla)
        
        x_pos = events_in[0][i_event]//sample_space
        y_pos = events_in[1][i_event]//sample_space
        p = events_in[3][i_event]
        data[int(x_pos), int(y_pos), int(p)] = 1.

        if i_event % sample_events == sample_events//2 :
            if debug:
                print(f'DEBUG {i_event} {i_event//sample_events} ')
                print(f'DEBUG {y[i_event//sample_events]}   ')
                print(f'DEBUG  {events_in[-3][i_event]} ')
            X[i_event//sample_events, :] = data.ravel()
            y[i_event//sample_events] = events_in[-3][i_event]
            
    if verbose: print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))
    
    # one hot encoding
    #y_out = np.zeros([y.shape[0],int(max(y)+1)])
    #for i in range(y.shape[0]):
    #    y_out[i,int(y[i])]=1
    
    return X, y#_out

def get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test):
    hotshom = network(krnlinit=krnlinit, tau=tau, nblay=nblay, nbclust=nbclust, homeo=homeo, sigma=sigma, homeinv=homeinv, jitter=jitter)
    arch = [hotshom.L[i].kernel.shape[1] for i in range(len(hotshom.L))]
    fname = f'{records_path}/EXP_03_NMNIST/{timestr}_hots_{dataset}_{arch}_{tau}_{homeo}_{sigma}_{homeinv}_{jitter}'
    print(fname)
    if not os.path.isfile(fname+'_model.pkl'):
        loaderhom, order = hotshom.learning1by1(dataset=dataset)
        with open(fname+'_model.pkl', 'wb') as file:
            pickle.dump([hotshom, loaderhom, order], file, pickle.HIGHEST_PROTOCOL)
    else:
        with open(fname+'_model.pkl', 'rb') as file:
            hotshom, loaderhom, order = pickle.load(file)
            
    if not os.path.isfile(fname+f'_evout_{nb_train}_{nb_test}.pkl'):       
        _, loaderhom, out_train = hotshom.running(loaderhom, order, nb_digit = nb_train, LR=True)
        _, loaderhom, out_test = hotshom.running(loaderhom, order, nb_digit = nb_test, LR=True)
        with open(fname+f'_evout_{nb_train}_{nb_test}.pkl', 'wb') as file:
            pickle.dump([out_train, out_test], file, pickle.HIGHEST_PROTOCOL)
    else:
        with open(fname+f'_evout_{nb_train}_{nb_test}.pkl', 'rb') as file:
            out_train, out_test = pickle.load(file)
    return out_train, out_test


# ### Performing logistic regression
#
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html
#

# from sklearn.linear_model import LogisticRegression as LR
#from sklearn.linear_model import LogisticRegressionCV as LR
# 
opts_LR = dict(max_iter=2000, # random_state=0,
               n_jobs=-1, class_weight='balanced')
#opts_LR['Cs'] = 5
opts_LR['Cs'] = 32
# TODO for a publication use 100 from 10^{-10} to 10

#from sklearn.model_selection import train_test_split

def tic():
    global ttic
    ttic = time.time()
def toc():
    print(f'Done in {time.time() - ttic:.3f} s')

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size
# https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html
class LogisticRegressionModel(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LogisticRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.LogSoftmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [ ]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 9 + 1
batch_size = 256
n_classes=10
amsgrad = False # gives similar results
amsgrad = True  # gives similar results

def fit_data(factors, y, 
            learning_rate=learning_rate,
            batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, **kwargs
        ):

    # print(factors.shape, y.shape)
    X, labels = torch.Tensor(factors), torch.Tensor(y).to(torch.long)
    # print(X.shape, labels.shape)
    loader = DataLoader(
        TensorDataset(X, labels), batch_size=batch_size, shuffle=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')
    
    N_batch = factors.shape[0]
    N = factors.shape[1]
    n_classes = 10 # y.shape[1]
    logistic_model = LogisticRegressionModel(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    for epoch in range(int(num_epochs)):
        logistic_model.train()
        losses = []
        for X_, labels_ in loader:
            X_, labels_ = X_.to(device), labels_.to(device)
            outputs = logistic_model(X_)
            #target = torch.argmax(torch.squeeze(labels_,dim=1), 1)
            loss = criterion(outputs, labels_)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")
            
    return logistic_model, losses

In [ ]:
tic()
events_train, events_test = get_nmnist(nb_train, nb_test)
toc()

In [ ]:
tic()
X_train, y_train = gather_data(events_train, verbose=verbose)
toc()

In [ ]:
y_train

In [ ]:
N = X_train.shape[1]

logistic_model = LogisticRegressionModel(N, n_classes)

logistic_model, losses = fit_data(X_train, y_train, verbose=True)
# print("Final mean loss =", np.mean(losses))

In [ ]:
def predict_data(factors, y, model,
            batch_size=batch_size,  # gamma=gamma,
            verbose=False, **kwargs
        ):
    X = torch.Tensor(factors)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'device -> {device}')

    logistic_model = model.to(device)
    
    X = X.to(device)
    outputs = logistic_model(X)

    pred_target = torch.argmax(outputs, dim=1)
    
    return pred_target

In [ ]:
X_test, y_test = gather_data(events_test, verbose=verbose)
pred_target = predict_data(X_test, y_test, logistic_model)

In [ ]:
y_test


In [ ]:
pred_target

In [ ]:
pred_target.detach().numpy() 

In [ ]:
pred_target.detach().numpy() == y_test

In [ ]:
np.mean(pred_target.detach().numpy() == y_test)

In [ ]:
print(f'Accuracy = {np.mean(pred_target.detach().numpy() == y_test)*100:.1f}%')

In [ ]:
events_train_o, events_test_o = get_events(tau, krnlinit, nblay, nbclust, homeo, records_path, timestr, dataset, sigma, homeinv, jitter, nb_train, nb_test)
X_train, y_train = gather_data(events_train_o, verbose=verbose)

In [ ]:
n_classes = 10
N = X_train.shape[1]

logistic_model = LogisticRegressionModel(N, n_classes)

logistic_model, loss = fit_data(X_train, y_train, verbose=True)
print("Final loss =", loss)

In [ ]:
X_test, y_test = gather_data(events_test_o, verbose=verbose)
pred_target = predict_data(X_test, y_test, logistic_model)